# Capture the Flag (RL - Policy Gradient)

- Seung Hyun Kim
- skim449@illinois.edu

## Notes
- This notebook includes:
    - Building the structure of policy driven network.
    - Training with/without render
    - Saver that save model and weights to ./model directory
    - Writer that will record some necessary datas to ./logs
- This notebook does not include running the CtF game with the RL policy. Using the network will be separately scripted in policy/policy_RL1.py.
    - cap_test.py is changed appropriately.
    
## References :
- https://github.com/awjuliani/DeepRL-Agents/blob/master/Vanilla-Policy.ipynb (source)
- https://www.youtube.com/watch?v=PDbXPBwOavc

In [1]:
!rm -rf logs/B1R4_AC/ model/B1R4_AC

In [2]:
TRAIN_NAME='B1R4_AC'
LOG_PATH='./logs/'+TRAIN_NAME
MODEL_PATH='./model/' + TRAIN_NAME
GPU_CAPACITY=0.125 # gpu capacity in percentage

In [3]:
import os

import tensorflow as tf
import tensorflow.contrib.slim as slim
import tensorflow.contrib.layers as layers
from tensorflow.python.client import device_lib
import matplotlib.pyplot as plt
%matplotlib inline

import time
import gym
import gym_cap
import gym_cap.envs.const as CONST
import numpy as np
import random

# the modules that you can use to generate the policy.
import policy.patrol 
import policy.random
import policy.simple # custon written policy
import policy.policy_RL
import policy.zeros

# Data Processing Module
from DataModule import one_hot_encoder, VISION_dX, VISION_dY

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
#device_lib.list_local_devices()

## Hyperparameters

In [5]:
# Training Related
total_episodes = 20000 #Set total number of episodes to train agent on.
max_ep = 300
update_frequency = 50
batch_size = 2000
experience_size=50000

# Saving Related
save_network_frequency = 1000
save_stat_frequency = update_frequency
moving_average_step = update_frequency

# General
LEARNING_RATE_FIX = True

## Environment Setting

In [6]:
if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_PATH)
    
#Create a directory to save episode playback gifs to
if not os.path.exists(LOG_PATH):
    os.makedirs(LOG_PATH)

In [7]:
env = gym.make("cap-v0") # initialize the environment
policy_red = policy.random.PolicyGen(env.get_map, env.get_team_red)
action_space = 5
n_agent = len(env.get_team_blue)
print('red number : ', len(env.get_team_red))
print('blue number : ', len(env.get_team_blue))

red number :  0
blue number :  1


## Discount Reward

In [8]:
gamma = 0.99

def discount_rewards(r):
    """ take 1D float array of rewards and compute discounted reward """
    discounted_r = np.zeros_like(r)
    running_add = 0.0
    for t in reversed(range(r.size)):
        running_add = (running_add * gamma + r[t])
        discounted_r[t] = running_add
    #discounted_r = (discounted_r - np.mean(discounted_r)) / (np.std(discounted_r)+1e-8) # normalize
    return discounted_r

## Network Setting

In [9]:
class agent():
    def __init__(self, lr_actor, lr_critic, in_size,action_size):
        if LEARNING_RATE_FIX:
            self.learning_rate_actor = tf.constant(lr_actor, tf.float32, name='learning_rate_actor')
            self.learning_rate_critic = tf.constant(lr_critic, tf.float32, name='learning_rate_critic')
        else:
            self.learning_rate_actor = tf.placeholder(tf.float32, shape=None, name='learning_rate_actor')
            self.learning_rate_critic = tf.placeholder(tf.float32, shape=None, name='learning_rate_critic')
        
        #These lines established the feed-forward part of the network. The agent takes a state and produces an action.
        self.state_input = tf.placeholder(shape=in_size,dtype=tf.float32, name='state')
        
        layer = slim.conv2d(self.state_input, 16, [5,5],# activation_fn=tf.nn.relu,
                            weights_initializer=layers.xavier_initializer_conv2d(),
                            biases_initializer=tf.zeros_initializer(),
                            padding='SAME',
                            scope='conv1')
        #layer = slim.avg_pool2d(layer, [2,2])
        layer = slim.conv2d(layer, 16, [3,3],# activation_fn=tf.nn.relu,
                            weights_initializer=layers.xavier_initializer_conv2d(),
                            biases_initializer=tf.zeros_initializer(),
                            padding='SAME',
                            scope='conv2')
        layer = slim.flatten(layer)
        #adv_stream, val_stream = tf.split(layer, num_or_size_splits=2)
        # Actor stream
        actor = slim.fully_connected(layer, 1024,
                                    weights_initializer=layers.xavier_initializer(),
                                    activation_fn=tf.nn.relu,
                                    biases_initializer=tf.zeros_initializer())
        self.dense = slim.fully_connected(actor, action_size, 
                                    weights_initializer=layers.xavier_initializer(),
                                    #activation_fn=tf.nn.relu,
                                    biases_initializer=tf.zeros_initializer(),
                                    scope='output_fc')
        self.output = tf.nn.softmax(self.dense, name='action')
        
        # Value stream
        self.critic = slim.fully_connected(layer, 1,
                                    weights_initializer=layers.xavier_initializer(),
                                    #activation_fn=tf.nn.relu,
                                    biases_initializer=tf.zeros_initializer(),
                                    scope='critic')
        
        tf.summary.histogram('output', self.output)
        
        with tf.name_scope('weights'):
            for var in slim.get_model_variables():
                tf.summary.histogram(var.op.name, var)

        #The next six lines establish the training proceedure. We feed the reward and chosen action into the network
        #to compute the loss, and use it to update the network.
    
        with tf.name_scope('holders'):
            self.action_holder = tf.placeholder(shape=[None],dtype=tf.int32)
            self.action_OH = tf.one_hot(self.action_holder, action_size)
            self.reward_holder = tf.placeholder(shape=[None],dtype=tf.float32)


        with tf.name_scope('Loss'):
            self.adv = self.reward_holder - tf.stop_gradient(tf.reduce_sum(self.critic, axis=1))
            
            self.responsible_outputs = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=self.dense, labels=self.action_holder)
            self.action_loss = tf.reduce_mean(self.responsible_outputs * self.adv)
            
            #self.responsible_outputs = tf.reduce_sum(self.output * self.action_OH, axis=1) #tf.gather(tf.reshape(self.output, [-1]), self.indexes) # output
            #self.action_loss = -tf.reduce_mean(tf.log(self.responsible_outputs)*self.adv)
            
            self.critic_loss = tf.reduce_mean(tf.squared_difference(self.reward_holder, tf.reduce_sum(self.critic, axis=1)))
            self.loss = self.action_loss + self.critic_loss
            
            tf.summary.scalar(name='loss', tensor=self.action_loss)
            tf.summary.scalar(name='critic_loss', tensor=self.critic_loss)
            tf.summary.scalar(name='AC_loss', tensor=self.loss)

        with tf.name_scope('optimizer'):
            optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate_actor)
            
        with tf.name_scope('gradients'):
            self.gradients = optimizer.compute_gradients(self.loss)
            #self.gradients = [tf.clip_by_value(grad, -5., 5.) for grad in self.gradients] # Clipping
            for var, grad in zip(slim.get_model_variables(), self.gradients):
                tf.summary.histogram(var.op.name+'/grad', grad[0])

        with tf.name_scope('update'):
            self.update_batch = optimizer.apply_gradients(self.gradients)

In [10]:
tf.reset_default_graph() # Clear the Tensorflow graph.
myAgent = agent(lr_actor=1e-5, lr_critic=1e-4,
                in_size=[None,VISION_dX,VISION_dY,6],action_size=5) #Load the agent.
global_step = tf.Variable(0, trainable=False, name='global_step') # global step
increment_global_step_op = tf.assign(global_step, global_step+1)
merged = tf.summary.merge_all()

## Session

In [ ]:
# Launch the session
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=GPU_CAPACITY, allow_growth=True)

sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
#sess = tf.Session()

total_reward = []
total_length = []
total_captured = []

# Setup Save and Restore Network
saver = tf.train.Saver(tf.global_variables())
writer = tf.summary.FileWriter(LOG_PATH, sess.graph)

ckpt = tf.train.get_checkpoint_state(MODEL_PATH)
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
    print("Load Model : ", ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())
    print("Initialized Variables")

Initialized Variables


In [ ]:
def record(summary_):
    with tf.device('/cpu:0'): 
        summary = tf.Summary()
        summary.value.add(tag='Records/mean_reward', simple_value=np.mean(total_reward[-moving_average_step:]))
        summary.value.add(tag='Records/mean_length', simple_value=np.mean(total_length[-moving_average_step:]))
        summary.value.add(tag='Records/mean_succeed', simple_value=np.mean(total_captured[-moving_average_step:]))
        writer.add_summary(summary, sess.run(global_step))
        
        #summary_str = sess.run(merged,feed_dict={myAgent.state_input:obs})
        writer.add_summary(summary_, sess.run(global_step))
        
        writer.flush()

In [ ]:
class Experience_buffer():
    def __init__(self, buffer_size = experience_size):
        self.buffer = []
        self.buffer_size = buffer_size
    
    def __len__(self):
        return len(self.buffer)
    
    def add(self,experience):
        if len(self.buffer) + len(experience) >= self.buffer_size:
            self.buffer[0:(len(experience)+len(self.buffer))-self.buffer_size] = []
        self.buffer.extend(experience)
    
    def flush(self):
        self.buffer = []
    
    def empty(self):
        return len(self.buffer)==0
    
    def sample(self, size=2000):
        if size > len(self.buffer):
            return np.array(self.buffer)
        else:
            return np.reshape(np.array(random.sample(self.buffer,size)),[size,4])

In [ ]:
def policy_rollout(DETERMINISTIC=True):
    # Run single episode, return the results
    s = env.reset(map_size=20, policy_red=policy_red)
    #obs = one_hot_encoder(s, env.get_team_blue) # partial observation
    obs_next = one_hot_encoder(env._env, env.get_team_blue)
    
    ep_history = []
    indv_history = [[] for _ in range(len(env.get_team_blue))]
    was_alive = [ag.isAlive for ag in env.get_team_blue]
    
    prev_reward=0
    frame=0
    for frame in range(max_ep+1):
        obs = obs_next
        
        with tf.device('/cpu:0'):
            act_prob = sess.run(myAgent.output, feed_dict={myAgent.state_input:obs})
        if DETERMINISTIC:
            act = np.argmax(act_prob, axis=1).tolist()
        else:
            #Probabilistically pick an action given our network outputs.
            act = [np.random.choice(action_space, p=act_prob[x]/sum(act_prob[x])) for x in range(n_agent)] # divide by sum : normalize
            
        s,r1,d,_ = env.step(act) #Get our reward for taking an action given a bandit.

        r = r1-prev_reward
        
        # If frame is at max and the game is still not done, negative reward
        if frame == max_ep and d == False:
            r -= 30
        #r -= 0.01*frame

        if d:
            critic_value = 0.0
        else:
            obs_next = one_hot_encoder(env._env, env.get_team_blue) # Full Observation
            critic_value = sess.run(myAgent.critic, feed_dict={myAgent.state_input:obs_next})
        
        td_targ = r + (0.9)*critic_value
        
        # Push history for individual that 'was' alive previous frame
        for idx, agent in enumerate(env.get_team_blue):
            if was_alive[idx]: indv_history[idx].append([obs[idx],act[idx],r,td_targ])
      
        # State Transition
        frame += 1
        prev_reward = r1
        was_alive = [ag.isAlive for ag in env.get_team_blue]
        
        if d:
            break

    for idx, history in enumerate(indv_history):
        if len(history)==0: continue
        _history = np.array(history)
        _history[:,2] = discount_rewards(_history[:,2])
        ep_history.extend(_history)
            
    if len(ep_history) > 0:        
        ep_history = np.stack(ep_history)
    
    return [frame, ep_history, r1, env.game_won, obs]

## Training

In [ ]:
if __name__=='__main__':
    ep = 0

    exp_buffer = Experience_buffer()
    try:
        progbar = tf.keras.utils.Progbar(total_episodes,width=5)
        while ep < total_episodes+1:
            # Run episode
            frame, history, reward, did_won, obs = policy_rollout()

            # Add history
            exp_buffer.add(history)

            if ep % update_frequency == 0 and ep != 0 and not exp_buffer.empty():
                batch_history = exp_buffer.sample(batch_size) # Sample from experience replay
                feed_dict={myAgent.reward_holder:batch_history[:,2],
                           myAgent.action_holder:batch_history[:,1],
                           myAgent.state_input:np.stack(batch_history[:,0])}
                with tf.device('/gpu:0'):
                    summary_, _ = sess.run([merged, myAgent.update_batch], feed_dict=feed_dict)
                if ep % save_stat_frequency == 0 and ep != 0:
                    record(summary_)
                exp_buffer.flush()

            total_reward.append(reward)
            total_length.append(frame)
            total_captured.append(env.game_won)
            
            # save every 100 ep
            if ep % save_network_frequency == 0 and ep != 0:
                #print(' Average r : ', np.mean(total_reward[-save_network_frequency:]))
                saver.save(sess, MODEL_PATH+'/ctf_policy.ckpt', global_step=global_step)
                #print("save weights: ", sess.run(global_step), 'episodes')

            ep += 1
            progbar.update(ep) # update progress bar
            sess.run(increment_global_step_op)

    except KeyboardInterrupt:
        print('\n\nManually stopped the training (KeyboardInterrupt)');
        plt.plot(total_reward)
        plt.figure()
        plt.plot(total_length)
        plt.figure()
        plt.plot(total_captured)
        saver.save(sess, MODEL_PATH+'/ctf_policy.ckpt', global_step=global_step)
        record(obs)
        print("save: ", sess.run(global_step), 'episodes')

   47/20000 [.....] - ETA: 9:24:56